In [ ]:
from common import *
from snobedo.snotel import CsvParser, SnotelLocations
from snobedo.lib.isnobal_helpers import day_filter

In [ ]:
client = start_cluster(4, 6)
client

In [ ]:
year = 2020
water_year = f'wy{year}'

melt_start = pd.to_datetime(f'{year}-04-01')

In [ ]:
snotel_sites = SnotelLocations()
snotel_sites.load_from_json(SNOTEL_DIR / 'site-locations/snotel_sites_2x2.json')

## SNOTEL

In [ ]:
schofield_snotel_csv = CsvParser.file(
    SNOTEL_DIR / water_year / f'Schofield/usda-csv/{year}-Schofield-Pass.csv'
)

butte_snotel_csv = CsvParser.file(
    SNOTEL_DIR / water_year / f'Butte/usda-csv/{year}-Butte.csv',
)

taylor_snotel_csv = CsvParser.file(
    SNOTEL_DIR / water_year / f'Upper-Taylor/usda-csv/{year}-Upper-Taylor.csv',
)

## iSnobal 

In [ ]:
wy_precip = xr.open_mfdataset(
    (SNOBAL_DIR / water_year / 'data/data20*/smrfOutputs/precip.nc').as_posix(),
    preprocess=day_filter,
    parallel=True,
)

In [ ]:
RESAMPLE_OPTS=dict(time='1D', base=23, keep_attrs=True)

butte_snobal = wy_precip.sel(
    x=snotel_sites.Butte.lon, y=snotel_sites.Butte.lat
).resample(**RESAMPLE_OPTS).sum().compute()

schofield_snobal = wy_precip.sel(
    x=snotel_sites.Schofield.lon, y=snotel_sites.Schofield.lat
).resample(**RESAMPLE_OPTS).sum().compute()
taylor_snobal = wy_precip.sel(
    x=snotel_sites.Taylor.lon, y=snotel_sites.Taylor.lat
).resample(**RESAMPLE_OPTS).sum().compute()

In [ ]:
plot_data = {
    'Butte': {
        'Snobal': butte_snobal,
        'Snotel': butte_snotel_csv,
    },
    'Schofield Pass': {
        'Snobal': schofield_snobal,
        'Snotel': schofield_snotel_csv,
    },
    'Upper Taylor': {
        'Snobal': taylor_snobal,
        'Snotel': taylor_snotel_csv,
    },
}

In [ ]:
figure_opts = dict(figsize=(8,4), dpi=200, tight_layout=True)
plot_range = pd.date_range(start=f'{year - 1}-10-01', periods=11, freq='MS')
text_props = dict(facecolor='whitesmoke', alpha=0.5, pad=.5, boxstyle='round')

xTicks = mdates.DateFormatter('%d-%b')
LABEL = 'Precipitation (mm)'

def plot_variable(save_figure=False):
    for site in plot_data.keys():
        snobal = plot_data[site]['Snobal']
        snotel = plot_data[site]['Snotel']
        
        figure, ax = plt.subplots(1, 1, **figure_opts)
        figure.set_facecolor('lightgrey')

        ax.bar(snobal.time, snotel[LABEL], color='steelblue', label='Snotel')
        snobal.precip.mean(['x', 'y']).plot(ax=ax, color='orange', label='Snobal', lw=1, alpha=0.8)

        ax.xaxis.set_major_formatter(xTicks)
        ax.set_xlim([plot_range[0], plot_range[-1]])
        ax.set_xlabel(f'Water Year {plot_range[-1].year}')
        ax.set_ylabel(LABEL)
        ax.set_title(f'Precipitation - {site}')
        ax.legend();
        
        if save_figure:
            filename = f'{year}_precipitation_{site}'.replace(' ', '-')
            plt.savefig(f"{FIGURES_DIR}/{filename}.png")

In [ ]:
plot_variable(True)